In [15]:
import glob
import re
import matplotlib as plt
import numpy as np
import xarray as xr
import pandas as pd

In [16]:
# !pip install dask_jobqueue

In [17]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [18]:
######## File paths ################
lustre_scratch    = "/glade/work/dpanta"
era5_surface_data = "/gdex/data/special_projects/harshah/ARCO/e5.oper.an.sfc"

In [19]:
cluster = PBSCluster(
        job_name = 'dask-osdf-25',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = lustre_scratch + '/dask/spill',
        log_directory = lustre_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        account='P43713000',
        walltime = '3:00:00',
        #interface = 'ib0'
        interface = 'ext'
    )

/glade/work/dpanta/conda-envs/conversion-src/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43303 instead
  warnings.warn(


In [20]:
client = Client(cluster)
# Scale the cluster and display cluster dashboard URL
n_workers =5
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)
cluster

PBSCluster(87b587a1, 'tcp://128.117.211.221:37517', workers=5, threads=5, memory=20.00 GiB)

## GMST functions

In [21]:
# GMST function ###
# calculate global means
def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    #other_dims = set(ds.dims) - {'time'}
    other_dims = set(ds.dims)
    return (ds * weight).mean(other_dims)




## Load data and compute GMST

In [22]:
# %%time
temp_2m = xr.open_zarr(era5_surface_data + '/e5.oper.an.sfc.2t.zarr')  
temp_2m = temp_2m.VAR_2T
# temp_2m

In [23]:
temp_2m_1950 = temp_2m.sel(time=temp_2m.time.dt.year==1949)
temp_2m_1950

<xarray.DataArray 'VAR_2T' (time: 8760, latitude: 721, longitude: 1440)> Size: 36GB
dask.array<getitem, shape=(8760, 721, 1440), dtype=float32, chunksize=(26, 480, 241), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 70kB 1949-01-01 ... 1949-12-31T23:00:00
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [24]:
np.sum(temp_2m_1950[0,:,:].values)

286054140.0

In [25]:
# %%time
# # Grid cells are spatially weighted and each time step gets the same weight 
gmst_1950 = global_mean(temp_2m_1950)
print(gmst_1950.values)

286.87586025340266


- This is sizeable dask graph!
- Let us try to compute it!

In [ ]:
%%timeit -r 3 
gmst_copy = gmst_1950.copy()
gmst_copy.values


In [ ]:
client.close()

In [ ]:
# 1950:array(286.76467424)
# 1945: 286.85706282

SyntaxError: illegal target for annotation (2689628681.py, line 2)